<a href="https://colab.research.google.com/github/ARBML/adawat/blob/main/notebooks/CAMeL_Tools_Guided_Tour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook provides a quick overview of the functionalities provided by [CAMeL Tools](https://github.com/CAMeL-Lab/camel_tools). While it doesn't cover every component in detail, it does provide a starting point for learning CAMeL Tools and navigating its APIs. For more detailed information please refer to the [CAMeL Tools documentation](https://camel-tools.readthedocs.io/en/latest/index.html).

# Arabic NLP Refresher

Before diving into CAMeL Tools, we recommend a quick refresher on Arabic NLP and the challenges it presents. [This webinar](https://wti.kaust.edu.sa/upcoming-events/Machine-Learning-Arabic-NLP-Webinar/ml-webinar-2020-keynote-1) presented by Dr. Nizar Habash provides a short introcuction on Arabic NLP as well as short examples of how CAMeL Tools can be used to solve various problems. A similar presentation of this webinar in Arabic is also available [here](https://www.youtube.com/watch?v=JQrmRvtx_VY).

*Note: The examples in this talk may not be compatible with current or newer versions of CAMeL Tools.*


# Installation and Setup

The following steps are needed if you want to run the examples in this notebook on Google Colaboratory. If you want to run this notebook on your own machine, please follow the [installation instructions](https://camel-tools.readthedocs.io/en/latest/getting_started.html#installation) instead.

First, we install the CAMeL Tools Python package.

In [ ]:
%pip install camel-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 13.9 MB/s 
     |████████████████████████████████| 175 kB 48.5 MB/s 
     |████████████████████████████████| 4.2 MB 64.0 MB/s 
     |████████████████████████████████| 418 kB 80.2 MB/s 
     |████████████████████████████████| 596 kB 72.4 MB/s 
     |████████████████████████████████| 6.6 MB 66.0 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
  Created wheel for camel-kenlm: filename=camel_kenlm-2021.12.27-cp37-cp37m-linux_x86_64.whl size=2336828 sha256=8e21a982a2f8e21e3d3fbdde4586994eb2ac9839a7eb25d503964a29006a2bdf
  Stored in directory: /root/.cache/pip/wheels/db/72/74/982f8c435f15b7feaf6dc8a03e212ff34e93f1f2d747059332
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=75a243fb9b03b9ac373101995004f5194b08aece26f5df4d9eada6f913b79fa3
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db

In order to use all the components provided in CAMeL Tools, we need to install all the datasets required by these components.
To do this in Colab, we need to first mount a Google Drive and create a directory where the data will be installed.

Run the code below and follow the instructions in the output.

In [ ]:
from google.colab import drive
import os

drive.mount('/gdrive')

%mkdir /gdrive/MyDrive/camel_tools

Mounted at /gdrive


Next, we need to tell CAMeL Tools to install the data in the newly created directory. This will take a couple of minutes to complete.

**NOTE:** You will need at least 2.3GB of available space on your Google Drive to install all the CAMeL Tools data.

In [ ]:
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

!export | camel_data -i all

The following packages will be installed: 'sentiment-analysis-mbert', 'disambig-ranking-cache-calima-glf-01', 'ner-arabert', 'sentiment-analysis-arabert', 'disambig-mle-calima-egy-r13', 'morphology-db-glf-01', 'dialectid-default', 'disambig-ranking-cache-calima-lev-01', 'disambig-mle-calima-msa-r13', 'morphology-db-msa-r13', 'disambig-ranking-cache-calima-egy-r13', 'disambig-bert-unfactored-glf', 'disambig-bert-unfactored-egy', 'disambig-bert-unfactored-msa', 'disambig-ranking-cache-calima-msa-r13', 'disambig-bert-unfactored-lev', 'morphology-db-egy-r13', 'morphology-db-lev-01'
Extracting package 'sentiment-analysis-mbert': 100% 712M/712M [00:03<00:00, 182MB/s]
Extracting package 'disambig-ranking-cache-calima-glf-01': 100% 21.5M/21.5M [00:00<00:00, 184MB/s]
Extracting package 'ner-arabert': 100% 542M/542M [00:02<00:00, 187MB/s]
Extracting package 'sentiment-analysis-arabert': 100% 542M/542M [00:02<00:00, 182MB/s]
Extracting package 'disambig-mle-calima-egy-r13': 100% 27.2M/27.2M [00:0

We also provide a lightweight dataset for the Morphology and Disambiguation components **only** that can be installed by calling `camel_data -i light` instead of `camel_data -i all`.


**Once the data has been installed on your Google Drive, you only need to run the following the next time you want to run this notebook.**

In [ ]:
%pip install camel-tools

from google.colab import drive
import os

drive.mount('/gdrive')
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Preprocessing

CAMeL Tools provides a suite of preprocessing utilities for cleaning and preparing Arabic text. Some of these may need to be used before passing text to other CAMeL Tools components.

## Simple Transliteration

Simple (or one-to-one) transliteration is the process of translating each Arabic character to/from a single non-Arabic character mapping (or encoding). The most common of these encoding schemes is the Buckwalter scheme and its variants that map Arabic characters to ASCII counterparts. You can see the various encoding schemes supported in CAMeL Tools and how they map to Arabic script [here](https://camel-tools.readthedocs.io/en/latest/reference/encoding_schemes.html).

This has many uses including:

- Smaller size of Arabic text on disk
- Allow non-Arabic speakers and people not familiar with Arabic script to read Arabic text
- Easier debugging by eliminating right-to-left display issues

Additionally, some Arabic NLP systems and resources use these transliteration schemes internally.

The first utility we provide to convert to/from different encodings is the [`CharMapper`](https://camel-tools.readthedocs.io/en/latest/api/utils/charmap.html). In the example below, we use a `CharMapper` to convert text in Arabic script to a Buckwalter encoding.

In [ ]:
from camel_tools.utils.charmap import CharMapper

sentence = "ذهبت إلى المكتبة."
print(sentence)

ar2bw = CharMapper.builtin_mapper('ar2bw')

sent_bw = ar2bw(sentence)
print(sent_bw)

ذهبت إلى المكتبة.
*hbt <lY Almktbp.


`CharMapper` is a very flexible component that can be used for many other types of text transformations and can be initialized with any character-to-string mapping. 

### Unicode Normalization

Text in Python 3 is represented as a series of [Unicode](https://unicode.org/standard/standard.html) characters. Some characters have different variants, for example the character 'ع' can also be represented as 'ﻊ', 'ﻋ', 'ﻌ' depending on where it appears in a word. For all these forms, we would call 'ع' their canonical form. Furthermore, some Unicode characters are a composition of multiple characters and in some cases can comprise entire phrases. For example, the character 'ﰷ' is a single Unicode character that represents the composition of 'ك' and 'ا'. Unicode decomposition is the process of splitting these composed characters into their individual constituents.

These variants and composed forms are generally used for display purposes and are problematic in text processing tasks. To convert Unicode text into something more suitable for text processing, we provide the [`normalize_unicode`](https://camel-tools.readthedocs.io/en/latest/api/utils/normalize.html#camel_tools.utils.normalize.normalize_unicode) function.

In the example below, we demonstrate a more extreme example of how `normalize_unicode` converts a composed character into its decomposed form.


In [ ]:
from camel_tools.utils.normalize import normalize_unicode

sentence = 'ﷺ'
print(sentence)

sent_norm = normalize_unicode(sentence)
print(sent_norm)

ﷺ
صلى الله عليه وسلم


**NOTE:** It is advised to run this function on all text prior to any further preprocessing or use.

### Orthographic Normalization

It is common for Arabic speakers to use shortcuts when typing Arabic text.
For example, the different variants of the letter alef ('ا', 'آ', 'أ', 'إ') may be typed as just 'ا'. Some of these substitutions can just be the result of typos. The presence of these variations can cause data sparsity and are usually normalized to a single form. Orthographic normalization is the process of converting letter variants or visually similar letters into a single form.

We provide a collection of orthographic normalization functions in CAMeL Tools. The example below demonstrates a few of them.

In [ ]:
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar

sentence = "هل ذهبت إلى المكتبة؟"
print(sentence)

# Normalize alef variants to 'ا'
sent_norm = normalize_alef_ar(sentence)
print(sent_norm)

# Normalize alef maksura 'ى' to yeh 'ي'
sent_norm = normalize_alef_maksura_ar(sent_norm)
print(sent_norm)

# Normalize teh marbuta 'ة' to heh 'ه'
sent_norm = normalize_teh_marbuta_ar(sent_norm)
print(sent_norm)

هل ذهبت إلى المكتبة؟
هل ذهبت الى المكتبة؟
هل ذهبت الي المكتبة؟
هل ذهبت الي المكتبه؟


The example above performs orthographic normalization on Arabic script. We provide variants of these functions for other encoding schemes as well (e.g. Buckwalter, Safe Buckwalter, etc). See [here](https://camel-tools.readthedocs.io/en/latest/api/utils/normalize.html) for more information.


## Dediacritization

Dediacritization is the process of removing Arabic diacritical marks. Diacritics increase data sparsity and so most Arabic NLP techniques ignore them. The example below shows how diacritics can be removed from Arabic text using the [`dediac_ar`](https://camel-tools.readthedocs.io/en/latest/api/utils/dediac.html#camel_tools.utils.dediac.dediac_ar) function:

In [ ]:
from camel_tools.utils.dediac import dediac_ar

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_dediac = dediac_ar(sentence)
print(sent_dediac)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
هل ذهبت إلى المكتبة؟


We provide dediacritization functions for other encoding schemes (eg. Buckwalter, Safe Buckwalter, etc) which you can learn more about [here](https://camel-tools.readthedocs.io/en/latest/api/utils/dediac.html).


## Word Tokenization

Some CAMeL Tools components expect text to be pretokenized by whitespace and punctuation. This means that an input sentence should be an array of words and punctuation instead of a single string. This is to preserve the alignment of inputs and outputs.

Python does provide the `split()` method to tokenize words by whitespace but it doesn't seperate punctuation from words. For example:

In [ ]:
sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = sentence.split()
print(sent_split)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ؟']


To split by whitespace and seperate punctuation, we currently provide the utility function [`simple_word_tokenize`](https://camel-tools.readthedocs.io/en/latest/api/tokenizers/word.html#camel_tools.tokenizers.word.simple_word_tokenize). The example below is similar to the one above, but this time we use `simple_word_tokenize()` instead of `split()`.

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = simple_word_tokenize(sentence)
print(sent_split)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ', '؟']


**NOTE:** `simple_word_tokenize`, as the name suggests is very simple and does not do anything fancy. It will seperate all punctuation regardless of context (ie. hashtags, emails, abbreviations, etc.)

# Morphology

CAMeL Tools provides a powerful morphological analysis, generation and reinflection system. This system is powered by [morphological databases](https://camel-tools.readthedocs.io/en/latest/api/morphology/database.html#databases) that are comprised of lexicons and compatibility tables. See [here](https://www.aclweb.org/anthology/W18-5816.pdf) for more details.

## Analysis


Morphological analysis is the process of generating all possible readings (analyses) of a given word out of context. All analyses are generated from the undiacritized form of the input word. Each of these analyses is defined by a set lexical and morphological features. 

In [ ]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer

# First, we need to load a morphological database.
# Here, we load the default database which is used for analyzing
# Modern Standard Arabic. 
db = MorphologyDB.builtin_db()

analyzer = Analyzer(db)

analyses = analyzer.analyze('موظف')

for analysis in analyses:
    print(analysis, '\n')

{'diac': 'مُوَظَّف', 'lex': 'مُوَظَّف', 'bw': 'مُوَظَّف/ADJ', 'gloss': 'employed;hired', 'pos': 'adj', 'prc3': '0', 'prc2': '0', 'prc1': '0', 'prc0': '0', 'per': 'na', 'asp': 'na', 'vox': 'na', 'mod': 'na', 'stt': 'i', 'cas': 'u', 'enc0': '0', 'rat': 'n', 'source': 'lex', 'form_gen': 'm', 'form_num': 's', 'd3seg': 'مُوَظَّف', 'caphi': 'm_u_w_a_dh._dh._a_f', 'd1tok': 'مُوَظَّف', 'd2tok': 'مُوَظَّف', 'pos_logprob': -0.9868824, 'd3tok': 'مُوَظَّف', 'd2seg': 'مُوَظَّف', 'pos_lex_logprob': -5.400551, 'num': 's', 'ud': 'ADJ', 'gen': 'm', 'catib6': 'NOM', 'root': '#.ظ.ف', 'bwtok': 'مُوَظَّف', 'pattern': 'مُوَ2َّ3', 'lex_logprob': -5.400551, 'atbtok': 'مُوَظَّف', 'atbseg': 'مُوَظَّف', 'd1seg': 'مُوَظَّف', 'stem': 'مُوَظَّف', 'stemgloss': 'employed;hired', 'stemcat': 'N-ap'} 

{'diac': 'مُوَظَّفَ', 'lex': 'مُوَظَّف', 'bw': 'مُوَظَّف/ADJ+َ/CASE_DEF_ACC', 'gloss': 'employed;hired+[def.acc.]', 'pos': 'adj', 'prc3': '0', 'prc2': '0', 'prc1': '0', 'prc0': '0', 'per': 'na', 'asp': 'na', 'vox': 'na', 

Let's examine just one of the output analyses generated from another word ('وسيكتبونها') to get a better idea of the verious morphological and lexical features that we provide. We provide a [handy reference page](https://camel-tools.readthedocs.io/en/latest/reference/camel_morphology_features.html) that explains what these features are and what their values mean.

```
diac: وَسَيَكْتُبُونَها
lex: كَتَب-u_1
bw: وَ/CONJ+سَ/FUT_PART+يَ/IV3MP+كْتُب/IV+ُونَ/IVSUFF_SUBJ:MP_MOOD:I+ها/IVSUFF_DO:3FS
gloss: and_+_will_+_they_(people)+write+it;them;her
pos: verb
prc3: 0
prc2: wa_conj
prc1: sa_fut
prc0: 0
per: 3
asp: i
vox: a
mod: i
stt: na
cas: na
enc0: 3fs_dobj
rat: n
source: lex
form_gen: m
form_num: p
pattern: وَسَيَ1ْ2ُ3ُونَها
root: ك.ت.ب
catib6: PRT+PRT+VRB+NOM
ud: CONJ+AUX+VERB+PRON
d1seg: وَ+_سَيَكْتُبُونَها
d1tok: وَ+_سَيَكْتُبُونَها
atbseg: وَ+_سَ+_يَكْتُبُونَ_+ها
d3seg: وَ+_سَ+_يَكْتُبُونَ_+ها
d2seg: وَ+_سَ+_يَكْتُبُونَها
d2tok: وَ+_سَ+_يَكْتُبُونَها
atbtok: وَ+_سَ+_يَكْتُبُونَ_+ها
d3tok: وَ+_سَ+_يَكْتُبُونَ_+ها
bwtok: وَ+_سَ+_يَ+_كْتُب_+ُونَ_+ها
pos_lex_logprob: -3.648503
caphi: w_a_s_a_y_a_k_t_u_b_uu_n_a_h_aa
pos_logprob: -1.023208
gen: m
lex_logprob: -3.648503
num: p
stem: كْتُب
stemgloss: write
stemcat: IV
```

## Generation

Generation is the process of inflecting a lemma
for a set of morphological features. The example below generates all the possible inflections for the lemma 'مُوَظَّف' (employee) that are masculine plural nouns.

In [ ]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.generator import Generator

# We need to indicate that the database we are loading will be
# used for generation.
db = MorphologyDB.builtin_db(flags='g')

generator = Generator(db)

lemma = 'مُوَظَّف'
features = {
    'pos': 'noun',
    'gen': 'm',
    'num': 'p'
}

analyses = generator.generate(lemma, features)

# Extract and print unique diacritizations from generated analyses
for diac in set([a['diac'] for a in analyses]):
    print(diac)

مُوَظَّفِينَ
مُوَظَّفِي
مُوَظَّفُونَ
مُوَظَّفُو


**NOTE:** `'pos'` is the only *required* feature that needs to be specified.

## Reinflection

Reinflection is the process of converting a given word in any form to a different form (i.e. tense, gender, etc). The CAMeL Tools reinflector works similar to the generator except that the word doesn't have to be a lemma and it is not have to be restricted to a specific `'pos'`. The example below reinflects the word 'شوارع' (streets) into its dual form with a proclitic 'ب'.

In [ ]:
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.reinflector import Reinflector

# We need to indicate that the database we are loading will be
# used for reinflection.
db = MorphologyDB.builtin_db(flags='r')

reinflector = Reinflector(db)

word = 'شوارع'
features = {
    'num': 'd',
    'prc1': 'bi_prep'
}

analyses = reinflector.reinflect(word, features)

# Extract and print unique diacritizations from reinflected analyses
for diac in set([a['diac'] for a in analyses]):
    print(diac)

بِشارِعَيْنِ
بِشارِعَيْ


# Disambiguation

Disambiguation is the process of determining what is the most likely analysis of a word in a given context. In CAMeL Tools, disambiguation is the backbone for many Arabic NLP tasks such as diacritization, POS tagging and morphological tokenization. At the moment, we provide a disambiguator that uses a Maximum Likelihood Estimation model, [`MLEDisambiguator`](https://camel-tools.readthedocs.io/en/latest/api/disambig/mle.html#camel_tools.disambig.mle.MLEDisambiguator). More advanced disambiguation techniques will be added to CAMeL Tools in the near future.

In the example below, we use a disambiguator to compute the most likely diacritization, POS tag, and lemma for all words in a given sentence.

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator

mle = MLEDisambiguator.pretrained()

# The disambiguator expects pre-tokenized text
sentence = simple_word_tokenize('نجح بايدن في الانتخابات')

disambig = mle.disambiguate(sentence)

# For each disambiguated word d in disambig, d.analyses is a list of analyses
# sorted from most likely to least likely. Therefore, d.analyses[0] would
# be the most likely analysis for a given word. Below we extract different
# features from the top analysis of each disambiguated word into seperate lists.
diacritized = [d.analyses[0].analysis['diac'] for d in disambig]
pos_tags = [d.analyses[0].analysis['pos'] for d in disambig]
lemmas = [d.analyses[0].analysis['lex'] for d in disambig]

# Print the combined feature values extracted above
for triplet in zip(diacritized, pos_tags, lemmas):
    print(triplet)


('نَجَحَ', 'verb', 'نَجَح')
('بايدن', 'noun_prop', 'بايدن')
('فِي', 'prep', 'فِي')
('الاِنْتِخاباتِ', 'noun', 'ٱِنْتِخاب')


# Tagging

The disambiguation example above is not very readable due to the amount of list and dictionary dereferencing just to extract a feature.
Additionally, for general tagging applications, extracting a feature from the top analysis is not sufficient.
There are some edge cases that need to be handled. For example, a word could return no analyses at all if the word isn't in the morphological database. We would then need to check the result of each disambiguated word to see if an analysis exists and then output some default value if not. Furthermore, the default value would differ depending on what feature we are tagging.

To handle these edge cases and to simplify the usage of a disambiguator, we provide the [`DefaultTagger`](https://camel-tools.readthedocs.io/en/latest/api/tagger/default.html#camel_tools.tagger.default.DefaultTagger) class which wraps a disambiguator and handles edge cases by providing sensible default values. The example below shows 

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mle = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mle, 'pos')

# The tagger expects pre-tokenized text
sentence = simple_word_tokenize('نجح بايدن في الانتخابات')

pos_tags = tagger.tag(sentence)

print(pos_tags)

['verb', 'noun_prop', 'prep', 'noun']


# Morphological Tokenization

Previously we discussed word tokenization as the process of splitting sentences by whitspace and seperating punctuation. Another type of tokenization is morphological tokenization whereby Arabic words are split into component prefixes, stems, and suffixes.

There are also different tokenization schemes for Arabic each having different use cases.

We provide the [`MorphologicalTokenizer`](https://camel-tools.readthedocs.io/en/latest/api/tokenizers/morphological.html#camel_tools.tokenizers.morphological.MorphologicalTokenizer) class to tokenize words in different schemes. It behaves very much like the `DefaultTagger` in that it uses a disambiguator to first disambiguate words and then extracts a particular tokenization feature, but it has the following differences:

- While the `DefaultTagger` produces exactly one output for each input word, the `MorphologicalTokenizer` might produce multiple output tokens.
-  The `MorphologicalTokenizer` can be configured to produce diacritized and undiacritized output.

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer

# The tokenizer expects pre-tokenized text
sentence = simple_word_tokenize('فتنفست الصعداء')
print(sentence)

# Load a pretrained disambiguator to use with a tokenizer
mle = MLEDisambiguator.pretrained('calima-msa-r13')

# Without providing additional arguments, the tokenizer will output undiacritized
# morphological tokens for each input word delimited by an underscore.
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok')
tokens = tokenizer.tokenize(sentence)
print(tokens)

# By specifying `split=True`, the morphological tokens are output as seperate
# strings.
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)
tokens = tokenizer.tokenize(sentence)
print(tokens)

# We can output diacritized tokens by setting `diac=True`
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True, diac=True)
tokens = tokenizer.tokenize(sentence)
print(tokens)

['فتنفست', 'الصعداء']
['ف+_تنفست', 'ال+_صعداء']
['ف+', 'تنفست', 'ال+', 'صعداء']
['فَ+', 'تَنَفَّسْتُ', 'ال+', 'صُعَداءَ']


We can see a list of supported tokenization schemes by calling the `mle.tok_feats()`.

**NOTE:** Different morphological databases will have differing support for each of these tokenization schemes.

In [ ]:
print(list(mle.tok_feats()))

['d2tok', 'atbtok', 'd3seg', 'bwtok', 'atbseg', 'd3tok', 'd2seg', 'd1seg', 'd1tok']


Try running the previous example using a different scheme than 'd3tok' to see how the output tokenization might vary.

# Dialect Identification

We provide a pretrained dialect identification system that can distinguish between 25 city dialects as well as Modern Standard Arabic. The model can be accessed using the [`DialectIdentifier`](https://camel-tools.readthedocs.io/en/latest/api/dialectid.html#camel_tools.dialectid.DialectIdentifier) class. In addition to city dialects, we can provide results aggregated by region and by country. While these agregated results are less fine-grained, they tend to be more accurate.

The example below illustrates how `DialectIdentifier` can be used to predict the dialects of given sentences by city, country and region. 

**NOTE:** You may get some warnings when running the example below in Colab. These can be safely ignored.

In [ ]:
from camel_tools.dialectid import DialectIdentifier

did = DialectIdentifier.pretrained()

sentences = [
    'مال الهوى و مالي شكون اللي جابني ليك  ما كنت انايا ف حالي بلاو قلبي يانا بيك',
    'بدي دوب قلي قلي بجنون بحبك انا مجنون ما بنسى حبك يوم'
]

predictions = did.predict(sentences, 'city')
print([p.top for p in predictions])

predictions = did.predict(sentences, 'country')
print([p.top for p in predictions])

predictions = did.predict(sentences, 'region')
print([p.top for p in predictions])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own

['Rabat', 'Aleppo']
['Morocco', 'Syria']
['Maghreb', 'Levant']


# Sentiment Analysis

We provide a pretrained [sentiment analysis](https://en.wikipedia.org/wiki/Sentiment_analysis) system that has been trained using a combination of multiple data sets. The model can be accessed using the [`SentimentAnalyzer`](https://camel-tools.readthedocs.io/en/latest/api/sentiment.html#camel_tools.sentiment.SentimentAnalyzer) class and it outputs one of three sentiment tags for each sentence: `'positive'`, `'negative'` and `'neutral'`. The example below demonstrates how it can be used.

In [ ]:
from camel_tools.sentiment import SentimentAnalyzer

sa = SentimentAnalyzer.pretrained()

sentences = [
    'أنا بخير',
    'أنا لست بخير'
]

sentiments = sa.predict(sentences)

print(sentiments)

['positive', 'negative']


# Named Entitiy Recognition

CAMeL Tools comes with an easy-to-use, pretrained [named-entitity recognition](https://en.wikipedia.org/wiki/Named-entity_recognition)  (NER) system that can be accessed using the [`NERecognizer`](https://camel-tools.readthedocs.io/en/latest/api/ner.html#camel_tools.ner.NERecognizer) class. 

For each token in an input sentence, `NERecognizer` outputs a label that indicates the type of named-entity.The system outputs one of the following labels for each token: `'B-LOC'`, `'B-ORG'`, `'B-PERS'`, `'B-MISC'`, `'I-LOC'`, `'I-ORG'`, `'I-PERS'`, `'I-MISC'`, `'O'`.
Named-entites can either be a `LOC` (location), `ORG` (organization), `PERS` (person), or `MISC` (miscallaneous).

Labels beginning with `B` indicate that their corresponding tokens are the begininging of a multi-word named-entity or is a single-token named-entity'. Those begining with `I` indicate that their corresponding tokens are continuations of a multi-word named-entity. Words that aren't named-entities are given the `'O'` label.

The example below illustrates how `NERecognizer` can be used to label named-entities in a given sentence.


In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.ner import NERecognizer

ner = NERecognizer.pretrained()

# NERecognizer expects pre-tokenized text
sentence = simple_word_tokenize('إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع.')

labels = ner.predict_sentence(sentence)

# Print each token paired with it's NER label
print(list(zip(sentence, labels)))

[('إمارة', 'O'), ('أبوظبي', 'B-LOC'), ('هي', 'O'), ('إحدى', 'O'), ('إمارات', 'O'), ('دولة', 'O'), ('الإمارات', 'B-LOC'), ('العربية', 'I-LOC'), ('المتحدة', 'I-LOC'), ('السبع', 'O'), ('.', 'O')]
